<a href="https://colab.research.google.com/github/KARTHICK20-jan/ENEC-Karthick/blob/main/Datanetra_AI_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import gradio as gr
from io import BytesIO
from sklearn.cluster import KMeans
import textwrap
import datetime

# ---------------- HELPERS ----------------
def normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-9)

# ---------------- INSIGHTS FUNCTION ----------------
def generate_insights(user_name, mobile_number, company_name, state, city, business_type, file):
    if file is None:
        return "Please upload an Excel file."

    df = pd.read_excel(file.name)

    # Scoring
    df["Cashflow_Stress"] = normalize(df["Monthly_Operating_Cost_INR"] / df["Monthly_Sales_INR"])
    df["Loan_Stress"] = normalize(df["Outstanding_Loan_INR"] / (df["Monthly_Sales_INR"]*12))
    df["Financial_Risk_Score"] = (0.5*df["Cashflow_Stress"] + 0.5*df["Loan_Stress"]).clip(0,1)

    df["Vendor_Score"] = (0.5*df["Vendor_Delivery_Reliability"] +
                          0.3*normalize(df["Inventory_Turnover"]) +
                          0.2*normalize(df["Avg_Margin_Percent"])) .clip(0,1)

    df["Growth_Potential_Score"] = (0.4*normalize(df["Monthly_Demand_Units"]) +
                                    0.35*normalize(df["Avg_Margin_Percent"]) +
                                    0.25*normalize(df.get("Digital_Ad_Spend_INR", pd.Series(0)))) .clip(0,1)

    df["MSME_Health_Score"] = ((1 - df["Financial_Risk_Score"])*0.4 +
                               df["Vendor_Score"]*0.3 +
                               df["Growth_Potential_Score"]*0.3)*100

    # Top SKUs
    top_skus = df.sort_values(by="Monthly_Sales_INR", ascending=False).head(5)[["SKU_Name","Monthly_Sales_INR","Monthly_Demand_Units"]]

    # Insights text
    insights = f"""
**DataNetra Insights for {user_name} – {company_name} ({business_type}) in {city}, {state}:**

- Average Financial Risk Score: {df["Financial_Risk_Score"].mean():.2f} (Higher indicates risk)
- Average Vendor Score: {df["Vendor_Score"].mean():.2f} (Higher indicates better vendor reliability)
- Average Growth Potential Score: {df["Growth_Potential_Score"].mean():.2f}
- Overall MSME Health Score: {df["MSME_Health_Score"].mean():.2f}%

**Top 5 SKUs:**
{top_skus.to_string(index=False)}

**Recommendations:**
- Focus on top SKUs for sales growth.
- Monitor high financial risk MSMEs carefully.
- Ensure reliable vendors are prioritized.
- Optimize inventory and demand planning.
"""
    return insights

# ---------------- PDF FUNCTION ----------------
def generate_pdf(user_name, mobile_number, company_name, state, city, business_type, file):
    if file is None:
        return None

    df = pd.read_excel(file.name)

    # Recalculate scores
    df["Cashflow_Stress"] = normalize(df["Monthly_Operating_Cost_INR"] / df["Monthly_Sales_INR"])
    df["Loan_Stress"] = normalize(df["Outstanding_Loan_INR"] / (df["Monthly_Sales_INR"]*12))
    df["Financial_Risk_Score"] = (0.5*df["Cashflow_Stress"] + 0.5*df["Loan_Stress"]).clip(0,1)
    df["Vendor_Score"] = (0.5*df["Vendor_Delivery_Reliability"] +
                          0.3*normalize(df["Inventory_Turnover"]) +
                          0.2*normalize(df["Avg_Margin_Percent"])) .clip(0,1)
    df["Growth_Potential_Score"] = (0.4*normalize(df["Monthly_Demand_Units"]) +
                                    0.35*normalize(df["Avg_Margin_Percent"]) +
                                    0.25*normalize(df.get("Digital_Ad_Spend_INR", pd.Series(0)))) .clip(0,1)
    df["MSME_Health_Score"] = ((1 - df["Financial_Risk_Score"])*0.4 +
                               df["Vendor_Score"]*0.3 +
                               df["Growth_Potential_Score"]*0.3)*100

    # Forecasts
    years = [1,2,3]
    sales_growth = 1 + (df["Growth_Potential_Score"].mean() * 0.15)
    demand_growth = 1 + (df["Growth_Potential_Score"].mean() * 0.12)
    avg_margin = df["Avg_Margin_Percent"].mean() / 100

    sales_forecast = [(df["Monthly_Sales_INR"].sum()*12)*(sales_growth**i) for i in years]
    demand_forecast = [df["Monthly_Demand_Units"].sum()*(demand_growth**i) for i in years]
    profit_forecast = [sales_forecast[i]*avg_margin for i in range(len(years))]

    # K-Means SKU segmentation
    kmeans_features = df[["Monthly_Demand_Units","Avg_Margin_Percent","Inventory_Turnover"]]
    kmeans = KMeans(n_clusters=3, random_state=42).fit(kmeans_features)
    df["SKU_Cluster"] = kmeans.labels_
    cluster_names = {0:"Cluster 0",1:"Cluster 1",2:"Cluster 2"}

    # Top SKUs
    top_skus = df.sort_values(by="Monthly_Sales_INR", ascending=False).head(5)[["SKU_Name","Monthly_Sales_INR","Monthly_Demand_Units"]]

    # PDF Generation
    pdf_buffer = BytesIO()
    with PdfPages(pdf_buffer) as pdf:
        # Cover Page
        plt.figure(figsize=(8,6)); plt.axis("off")
        plt.text(0.5,0.6,f"DataNetra – AI Decision Support Report", fontsize=18, ha='center')
        plt.text(0.5,0.5,f"User: {user_name}\nCompany: {company_name}\nBusiness: {business_type}\nLocation: {city}, {state}", fontsize=12, ha='center')
        plt.text(0.5,0.4,f"Date: {datetime.datetime.today().strftime('%d-%b-%Y')}", fontsize=10, ha='center')
        pdf.savefig(); plt.close()

        # Table of Contents
        toc = ["1. Executive Summary","2. Financial Risk Analysis","3. Vendor Analysis","4. Growth & Forecast",
               "5. SKU Segmentation","6. Actionable Insights & Conclusion"]
        plt.figure(figsize=(8,6)); plt.axis("off")
        y = 0.9
        plt.text(0.5,y,"Table of Contents", fontsize=16, ha='center')
        for i, item in enumerate(toc):
            y -= 0.1
            plt.text(0.1,y,f"{item} ...... Page {i+2}", fontsize=12)
        pdf.savefig(); plt.close()

        # Executive Summary
        plt.figure(figsize=(8,6)); plt.axis("off")
        summary_text = f"""Executive Summary:
- Overall MSME Health Score: {df["MSME_Health_Score"].mean():.2f}%
- Financial Risk Avg: {df["Financial_Risk_Score"].mean():.2f}
- Vendor Score Avg: {df["Vendor_Score"].mean():.2f}
- Growth Potential Avg: {df["Growth_Potential_Score"].mean():.2f}
"""
        plt.text(0.05,0.5,summary_text, fontsize=12)
        pdf.savefig(); plt.close()

        # Section 1: Financial Risk
        plt.figure(); plt.hist(df["Financial_Risk_Score"], bins=10, color="red")
        plt.title("Financial Risk Score Distribution", fontsize=14)
        plt.xlabel("Score"); plt.ylabel("Number of MSMEs"); plt.grid(True)
        plt.text(0.5, max(np.histogram(df["Financial_Risk_Score"], bins=10)[0])*0.8,
                 "Red: High financial risk → monitor closely", fontsize=10, color='black', ha='center')
        pdf.savefig(); plt.close()

        # Section 2: Vendor Analysis
        plt.figure(); plt.hist(df["Vendor_Score"], bins=10, color="green")
        plt.title("Vendor Score Distribution", fontsize=14)
        plt.xlabel("Score"); plt.ylabel("Number of Vendors"); plt.grid(True)
        plt.text(0.5, max(np.histogram(df["Vendor_Score"], bins=10)[0])*0.8,
                 "Green: Reliable vendor → safe to work with", fontsize=10, color='black', ha='center')
        pdf.savefig(); plt.close()

        # Section 3: Growth & Forecast
        # Sales Forecast
        plt.figure(); plt.plot([2026,2027,2028], sales_forecast, marker="o", color="blue")
        plt.title("3-Year Sales Forecast (INR)", fontsize=14)
        plt.xlabel("Year"); plt.ylabel("Total Sales (INR)"); plt.grid(True)
        plt.text(2026, min(sales_forecast)*0.95, f"Sales expected to grow from {sales_forecast[0]:,.0f} to {sales_forecast[-1]:,.0f}", fontsize=10)
        pdf.savefig(); plt.close()

        # Demand Forecast
        plt.figure(); plt.plot([2026,2027,2028], demand_forecast, marker="o", color="orange")
        plt.title("3-Year Demand Forecast (Units)", fontsize=14)
        plt.xlabel("Year"); plt.ylabel("Demand Units"); plt.grid(True)
        plt.text(2026, min(demand_forecast)*0.95, f"Demand projected from {int(demand_forecast[0])} to {int(demand_forecast[-1])} units", fontsize=10)
        pdf.savefig(); plt.close()

        # Profit Forecast
        plt.figure(); plt.plot([2026,2027,2028], profit_forecast, marker="o", color="purple")
        plt.title("3-Year Profit Forecast (INR)", fontsize=14)
        plt.xlabel("Year"); plt.ylabel("Profit (INR)"); plt.grid(True)
        plt.text(2026, min(profit_forecast)*0.95, f"Profit projected from {profit_forecast[0]:,.0f} to {profit_forecast[-1]:,.0f}", fontsize=10)
        pdf.savefig(); plt.close()

        # Section 4: SKU Segmentation
        plt.figure()
        colors = ["red","green","blue"]
        for cluster in df["SKU_Cluster"].unique():
            subset = df[df["SKU_Cluster"]==cluster]
            plt.scatter(subset["Monthly_Demand_Units"], subset["Avg_Margin_Percent"],
                        label=f"{cluster_names[cluster]}", color=colors[cluster])
        plt.title("SKU Segmentation (K-Means)", fontsize=14)
        plt.xlabel("Monthly Demand Units"); plt.ylabel("Average Margin %"); plt.legend()
        plt.text(max(df["Monthly_Demand_Units"])*0.5, max(df["Avg_Margin_Percent"])*0.95,
                 "Red/Green/Blue: SKU clusters → PUSH / HOLD / REDUCE", fontsize=10)
        pdf.savefig(); plt.close()

        # Top SKUs Table
        plt.figure(figsize=(8,2)); plt.axis("off")
        plt.text(0.01,0.5,"Top 5 Profitable SKUs:\n"+top_skus.to_string(index=False), fontsize=10)
        pdf.savefig(); plt.close()

        # Section 5: Conclusion & Actionable Insights
        plt.figure(figsize=(8,6)); plt.axis("off")
        conclusion_text = f"""
Conclusion & Actionable Insights:

Next 12 Months:
- Focus on top SKUs identified above for inventory and promotions.
- Monitor high financial risk MSMEs closely.
- Maintain reliable vendors; review HOLD SKUs monthly.
- Forecasted Sales: {sales_forecast[0]:,.0f} INR, Demand: {int(demand_forecast[0])} units, Profit: {profit_forecast[0]:,.0f} INR.

Next 3 Years:
- Sales expected to grow to {sales_forecast[-1]:,.0f} INR, Demand: {int(demand_forecast[-1])} units, Profit: {profit_forecast[-1]:,.0f} INR.
- PUSH SKUs: Allocate inventory and marketing spend.
- REDUCE SKUs: Minimize stock and consider promotions.
- HOLD SKUs: Monitor trends and adjust monthly.
"""
        plt.text(0.01,0.5,conclusion_text, fontsize=12)
        pdf.savefig(); plt.close()

    pdf_buffer.seek(0)

    # Save to temp file for download
    pdf_path = f"/tmp/{business_type}_{user_name}_report.pdf".replace(" ","_")
    with open(pdf_path, "wb") as f:
        f.write(pdf_buffer.getbuffer())

    return pdf_path

# ---------------- BUSINESS ASSISTANT ----------------
sample_prompts = [
    "Show top 5 vendors",
    "Projected sales for 2026",
    "Which SKUs to push?",
]

def business_assistant(file, query):
    if file is None or query.strip()=="":
        return "Upload file and type your query."
    df = pd.read_excel(file.name)
    df["Cashflow_Stress"] = normalize(df["Monthly_Operating_Cost_INR"] / df["Monthly_Sales_INR"])
    df["Loan_Stress"] = normalize(df["Outstanding_Loan_INR"] / (df["Monthly_Sales_INR"]*12))
    df["Financial_Risk_Score"] = (0.5*df["Cashflow_Stress"] + 0.5*df["Loan_Stress"]).clip(0,1)
    df["Vendor_Score"] = (0.5*df["Vendor_Delivery_Reliability"] +
                          0.3*normalize(df["Inventory_Turnover"]) +
                          0.2*normalize(df["Avg_Margin_Percent"])) .clip(0,1)
    df["Growth_Potential_Score"] = (0.4*normalize(df["Monthly_Demand_Units"]) +
                                    0.35*normalize(df["Avg_Margin_Percent"]) +
                                    0.25*normalize(df.get("Digital_Ad_Spend_INR", pd.Series(0)))) .clip(0,1)
    df["MSME_Health_Score"] = ((1 - df["Financial_Risk_Score"])*0.4 +
                               df["Vendor_Score"]*0.3 +
                               df["Growth_Potential_Score"]*0.3)*100
    if "top 5 vendors" in query.lower():
        top_vendors = df.sort_values("Vendor_Score", ascending=False).head(5)[["SKU_Name","Vendor_Score"]]
        return f"Top 5 Vendors:\n{top_vendors.to_string(index=False)}"
    if "projected sales" in query.lower():
        growth = 1 + df["Growth_Potential_Score"].mean()*0.15
        forecast = (df["Monthly_Sales_INR"].sum()*12)*growth
        return f"Projected sales for next year: {forecast:,.0f} INR"
    if "skus to push" in query.lower():
        kmeans_features = df[["Monthly_Demand_Units","Avg_Margin_Percent","Inventory_Turnover"]]
        kmeans = KMeans(n_clusters=3, random_state=42).fit(kmeans_features)
        df["SKU_Cluster"] = kmeans.labels_
        # Assume Cluster 0 = PUSH
        push_skus = df[df["SKU_Cluster"]==0][["SKU_Name","Monthly_Sales_INR"]]
        return f"PUSH SKUs:\n{push_skus.to_string(index=False)}"
    return "Query understood, but no specific logic applied."

# ---------------- GRADIO UI ----------------
states = ["Kerala","Tamil Nadu","Karnataka","Telangana","Andhra Pradesh","Delhi"]
cities = {
    "Kerala": ["Kochi", "Thiruvananthapuram", "Kozhikode"],
    "Tamil Nadu": ["Chennai", "Madurai", "Coimbatore", "Trichy"],
    "Karnataka": ["Bangalore", "Mysore", "Mangalore"],
    "Telangana": ["Hyderabad", "Warangal"],
    "Andhra Pradesh": ["Visakhapatnam", "Vijayawada", "Guntur"],
    "Delhi": ["New Delhi"]
}
business_types = ["Supermarket","Clothing","Pharmacy","Restaurant"]

with gr.Blocks() as demo:
    gr.HTML("""
    <div style='background: linear-gradient(to bottom, #FF9933 0%, #FFFFFF 50%, #138808 100%); padding:20px'>
        <h2 style='text-align:center;color:#000080'>India AI Innovation Challenge 2026 – DataNetra MSME Dashboard</h2>
    </div>
    """)

    with gr.Row():
        user_name = gr.Textbox(label="User Name")
        mobile_number = gr.Textbox(label="Mobile Number")
        company_name = gr.Textbox(label="Company / Business Type Name")

    with gr.Row():
        state = gr.Dropdown(label="State", choices=["Choose State"]+states, value="Choose State")
        city = gr.Dropdown(label="City", choices=["Choose City"], value="Choose City")
        business_type = gr.Dropdown(label="Business Type", choices=["Choose Business Type"]+business_types, value="Choose Business Type")

    file = gr.File(label="Upload Retail Data (Excel)")

    # Upload status message
    upload_status = gr.Textbox(label="Upload Status", interactive=False)

    # Generate Insights Button
    generate_insights_btn = gr.Button("Generate Insights", interactive=False)
    insights_box = gr.Textbox(label="AI Insights", lines=20)

    # PDF Report
    generate_pdf_btn = gr.Button("Generate PDF Report")
    pdf_output = gr.File(label="Download PDF Report")

    # File Upload Callbacks
    def file_uploaded_status(file, name):
        if file is not None and name:
            return f"Thank you {name} for uploading your data file! ✅ Now click Generate Insights."
        return ""

    def enable_insights_btn(file):
        if file is not None:
            return gr.update(interactive=True)
        return gr.update(interactive=False)

    file.upload(file_uploaded_status, inputs=[file, user_name], outputs=upload_status)
    file.upload(enable_insights_btn, inputs=file, outputs=generate_insights_btn)

    generate_insights_btn.click(generate_insights,
                                inputs=[user_name,mobile_number,company_name,state,city,business_type,file],
                                outputs=insights_box)

    generate_pdf_btn.click(generate_pdf,
                           inputs=[user_name,mobile_number,company_name,state,city,business_type,file],
                           outputs=pdf_output)

    # Business Assistant
    gr.HTML("<b>Ask Your Business Assistant (Press Enter or click 'Ask')</b>")
    gr.HTML("Example prompts: 'Show me top 5 vendors', 'Top SKUs to promote', 'Projected sales for 2026'")

    assistant_input = gr.Textbox(
        placeholder="Type your question here...",
        label="Ask Your Business Assistant",
        lines=1  # <-- single line to allow Enter key submission
    )
    assistant_output = gr.Textbox(label="Assistant Response", interactive=False, lines=8)
    assistant_btn = gr.Button("Ask")

    def ask_business_assistant(query, file):
        if not query.strip():
            return "Please ask a question related to the generated AI insights or report."

        q = query.lower()

        # --- INSIGHTS ---
        if "insight" in q or "summary" in q:
            return SESSION_CONTEXT.get("insights_text", "Please generate AI insights first.")

        # --- HEALTH ---
        if "health" in q:
            return f"Overall MSME Health Score: {SESSION_CONTEXT['summary'].get('health_score',0):.2f}%"

        # --- FINANCIAL RISK ---
        if "risk" in q:
            return f"Average Financial Risk Score: {SESSION_CONTEXT['summary'].get('financial_risk',0):.2f}"

        # --- TOP SKUS ---
        if "top sku" in q or "best sku" in q:
            skus = SESSION_CONTEXT.get("top_skus")
            if skus is None:
                return "Please generate insights first."
            return f"Top Performing SKUs:\n{skus.to_string(index=False)}"

        # --- SALES FORECAST ---
        if "sales" in q:
            f = SESSION_CONTEXT.get("forecasts", {})
            return f"Sales Forecast:\n2026: ₹{f.get('sales_2026',0):,.0f}\n2028: ₹{f.get('sales_2028',0):,.0f}"

        # --- DEMAND FORECAST ---
        if "demand" in q:
            f = SESSION_CONTEXT.get("forecasts", {})
            return f"Demand Forecast:\n2026: {int(f.get('demand_2026',0))} units\n2028: {int(f.get('demand_2028',0))} units"

        # --- PROFIT FORECAST ---
        if "profit" in q:
            f = SESSION_CONTEXT.get("forecasts", {})
            return f"Profit Forecast:\n2026: ₹{f.get('profit_2026',0):,.0f}\n2028: ₹{f.get('profit_2028',0):,.0f}"

        return "I can answer questions about insights, health score, top SKUs, sales, demand, and profit forecasts."

    # Button and Enter submission
    assistant_btn.click(ask_business_assistant, inputs=[assistant_input, file], outputs=assistant_output)
    assistant_input.submit(ask_business_assistant, inputs=[assistant_input, file], outputs=assistant_output)

    # City dropdown update
    def update_city_dropdown(s):
        if s in cities:
            return gr.update(choices=["Choose City"] + cities[s], value="Choose City")
        return gr.update(choices=["Choose City"], value="Choose City")
    state.change(update_city_dropdown, inputs=state, outputs=city)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f758ba1dddd9682882.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
